In [1]:
##구제시 년도별 레이어 만들기
import _util
import _config
import _codes
import pandas as pd

def create_table(year):
    sql = f'''
        do $$
        begin 
            if exists (select 1 from pg_tables where tablename = 'gujaesi_sgg_{year}_5179') then
            drop table gujaesi_sgg_{year}_5179 cascade;
            end if;

            if not exists (select 1 from pg_tables where tablename = 'gujaesi_sgg_{year}_5179') then 
            create table gujaesi_sgg_{year}_5179 as 
            with um as (
                select substring(s.sgg_cd, 1, 4) as sgg_cd_4, st_union(st_makevalid(s.geom)) as geom
                from simple_sgg_{year} s
                group by s.sgg_cd
            )
            select row_number() over() as fid
            , concat(substring(s.sgg_cd, 1, 4), 0) as sgg_cd
            -- , split_part(s.sgg_nm, ' ', 1) as sgg_nm
            , st_union(s.geom) as geom
            from simple_sgg_{year} s
            left join um on concat(um.sgg_cd_4, '0') = s.sgg_cd
            where um.geom is null
            group by substring(s.sgg_cd, 1, 4)
            -- split_part(s.sgg_nm, ' ', 1), substring(s.sgg_cd, 1, 4)
        --	union all 
        --	select row_number() over() as fid
        --	, concat(substring(s.sgg_cd, 1, 4), 0) as sgg_cd
        --	, split_part(s.sgg_nm, ' ', 1) as sgg_nm
        --	, s.geom as geom
        --	, s."year"
        --	from simple_sgg_{year} s
        --	left join um on concat(um.sgg_cd_4, '0') = s.sgg_cd
        --	where um.geom is not null
            ;
            end if;
        end $$;

        alter table gujaesi_sgg_{year}_5179
        alter column geom type geometry(geometry, 5179)
        using geom::geometry(geometry, 5179)
        ;
    '''

    _util.execute_sql(sql)


for year in _config.layer_years:
    create_table(year)


        do $$
        begin 
            if exists (select 1 from pg_tables where tablename = 'gujaesi_sgg_2000_5179') then
            drop table gujaesi_sgg_2000_5179 cascade;
            end if;

            if not exists (select 1 from pg_tables where tablename = 'gujaesi_sgg_2000_5179') then 
            create table gujaesi_sgg_2000_5179 as 
            with um as (
                select substring(s.sgg_cd, 1, 4) as sgg_cd_4, st_union(st_makevalid(s.geom)) as geom
                from simple_sgg_2000 s
                group by s.sgg_cd
            )
            select row_number() over() as fid
            , concat(substring(s.sgg_cd, 1, 4), 0) as sgg_cd
            -- , split_part(s.sgg_nm, ' ', 1) as sgg_nm
            , st_union(s.geom) as geom
            from simple_sgg_2000 s
            left join um on concat(um.sgg_cd_4, '0') = s.sgg_cd
            where um.geom is null
            group by substring(s.sgg_cd, 1, 4)
            -- split_part(s.sgg_nm, ' ', 1), substring

UndefinedTable: relation "simple_sgg_2000" does not exist
LINE 4:                 from simple_sgg_2000 s
                             ^
QUERY:  create table gujaesi_sgg_2000_5179 as 
            with um as (
                select substring(s.sgg_cd, 1, 4) as sgg_cd_4, st_union(st_makevalid(s.geom)) as geom
                from simple_sgg_2000 s
                group by s.sgg_cd
            )
            select row_number() over() as fid
            , concat(substring(s.sgg_cd, 1, 4), 0) as sgg_cd
            -- , split_part(s.sgg_nm, ' ', 1) as sgg_nm
            , st_union(s.geom) as geom
            from simple_sgg_2000 s
            left join um on concat(um.sgg_cd_4, '0') = s.sgg_cd
            where um.geom is null
            group by substring(s.sgg_cd, 1, 4)
            -- split_part(s.sgg_nm, ' ', 1), substring(s.sgg_cd, 1, 4)
        --	union all 
        --	select row_number() over() as fid
        --	, concat(substring(s.sgg_cd, 1, 4), 0) as sgg_cd
        --	, split_part(

In [5]:
##구제시 아닌 곳 레이어 만들기
import _util
import _config
import _codes
import pandas as pd

def create_table(year):
    sql = f'''
        do $$
        begin 
            if exists (select 1 from pg_tables where tablename = 'nogujaesi_sgg_{year}_5179') then
            drop table nogujaesi_sgg_{year}_5179 cascade;
            end if;

            if not exists (select 1 from pg_tables where tablename = 'nogujaesi_sgg_{year}_5179') then 
            create table nogujaesi_sgg_{year}_5179 as 
            with um as (
                select substring(s.sgg_cd, 1, 4) as sgg_cd_4, st_union(st_makevalid(s.geom)) as geom
                from simple_sgg_{year} s
                group by s.sgg_cd
            )
        	select row_number() over() as fid
        	, concat(substring(s.sgg_cd, 1, 4), 0) as sgg_cd
        	, split_part(s.sgg_nm, ' ', 1) as sgg_nm
        	, s.geom as geom
        	from simple_sgg_{year} s
        	left join um on concat(um.sgg_cd_4, '0') = s.sgg_cd
        	where um.geom is not null
            ;
            end if;
        end $$;

        alter table nogujaesi_sgg_{year}_5179
        alter column geom type geometry(geometry, 5179)
        using geom::geometry(geometry, 5179)
        ;
    '''

    _util.execute_sql(sql)


for year in _config.layer_years:
    create_table(year)


        do $$
        begin 
            if exists (select 1 from pg_tables where tablename = 'nogujaesi_sgg_2000_5179') then
            drop table nogujaesi_sgg_2000_5179 cascade;
            end if;

            if not exists (select 1 from pg_tables where tablename = 'nogujaesi_sgg_2000_5179') then 
            create table nogujaesi_sgg_2000_5179 as 
            with um as (
                select substring(s.sgg_cd, 1, 4) as sgg_cd_4, st_union(st_makevalid(s.geom)) as geom
                from simple_sgg_2000 s
                group by s.sgg_cd
            )
        	select row_number() over() as fid
        	, concat(substring(s.sgg_cd, 1, 4), 0) as sgg_cd
        	, split_part(s.sgg_nm, ' ', 1) as sgg_nm
        	, s.geom as geom
        	from simple_sgg_2000 s
        	left join um on concat(um.sgg_cd_4, '0') = s.sgg_cd
        	where um.geom is not null
            ;
            end if;
        end $$;

        alter table nogujaesi_sgg_2000_5179
        alter column geom ty

In [6]:
#지도 경량화 하기

import _util
import _config
import _codes
import pandas as pd

def create_table(year):
    year = int(year)
    if year < 2020: q1 = ', s."base_year" as "year"'
    else: q1 = ', substring(s."base_date", 1, 4) as "year'

    sql = f'''
        -- 지도 경량화하기
        do $$
        begin
            if exists (select 1 from pg_tables where tablename ='simple_sgg_{year}') then
            drop table simple_sgg_{year} cascade;
            end if;
            if not exists (select 1 from pg_tables where tablename = 'simple_sgg_{year}') then
            create table simple_sgg_{year} as 
            select id as fid
            , st_simplify(st_makevalid(s.geom), 0.9) AS geom
            , s."sigungu_cd" as sgg_cd
            , s."sigungu_nm" as sgg_nm
            {q1}
            from sgg_{year} s;
            end if;
        end $$;

        alter table simple_sgg_{year}
        alter column geom type geometry(geometry, 5179)
        using geom::geometry(geometry, 5179)
        ;
    '''

    _util.execute_sql(sql)

for year in _config.layer_years:
    create_table(year)



        -- 지도 경량화하기
        do $$
        begin
            if exists (select 1 from pg_tables where tablename ='simple_sgg_2000') then
            drop table simple_sgg_2000 cascade;
            end if;
            if not exists (select 1 from pg_tables where tablename = 'simple_sgg_2000') then
            create table simple_sgg_2000 as 
            select id as fid
            , st_simplify(st_makevalid(s.geom), 0.9) AS geom
            , s."sigungu_cd" as sgg_cd
            , s."sigungu_nm" as sgg_nm
            , s."base_year" as "year"
            from sgg_2000 s;
            end if;
        end $$;

        alter table simple_sgg_2000
        alter column geom type geometry(geometry, 5179)
        using geom::geometry(geometry, 5179)
        ;
    

        -- 지도 경량화하기
        do $$
        begin
            if exists (select 1 from pg_tables where tablename ='simple_sgg_2005') then
            drop table simple_sgg_2005 cascade;
            end if;
            if not exists (s

SyntaxError: unterminated quoted identifier at or near ""year
            from sgg_2020 s;
            end if;
        end "
LINE 14:             , substring(s."base_date", 1, 4) as "year
                                                         ^
